In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd() / 'torchcrepe'))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
import numpy as np
from IPython.display import Audio

In [ ]:
import torch
from torchaudio.transforms import GriffinLim
from torchvision.transforms import functional as tvf
from torchcrepe import predict
from torchcrepe.decode import argmax, weighted_argmax, viterbi

In [ ]:
PATH = '/home/kureta/Music/violin/Violin Samples/yee_amazing_grace.wav'

In [ ]:
SAMPLE_RATE = 44100
HOP_LENGTH = 512
N_FFT = 2048

# crepe hop length has to be the smallest power of 2 greater than `HOP_LENGTH * CREPE_SAMPLE_RATE / SAMPLE_RATE`
CREPE_HOP_LENGTH = 256
CREPE_SAMPLE_RATE = 16000

In [ ]:
y, sr = librosa.load(PATH, mono=False, sr=SAMPLE_RATE)

In [ ]:
s = np.abs(librosa.stft(y, n_fft=N_FFT, hop_length=HOP_LENGTH))
# zero out DC offset
s[:, 0, :] = 0.0

In [ ]:
librosa.display.specshow(librosa.amplitude_to_db(s[0]),
                         n_fft=N_FFT, sr=SAMPLE_RATE, hop_length=HOP_LENGTH, y_axis='linear', x_axis='time')
plt.show()

In [ ]:
pitches = predict(torch.from_numpy(y).cuda(), 16000, hop_length=256, decoder=viterbi, device='cuda')
pitches = tvf.resize(pitches.unsqueeze(1), [1, s.shape[2]]).squeeze(1)

In [ ]:
plt.plot(pitches.T.cpu())
plt.show()

In [ ]:
gl = GriffinLim(n_fft=N_FFT, hop_length=HOP_LENGTH, power=1.0).cuda()

In [ ]:
y_hat = gl(torch.from_numpy(s).cuda())

In [ ]:
Audio(y_hat.cpu(), rate=SAMPLE_RATE, normalize=False)